In [1]:
import pandas as pd
from datetime import datetime

rawpath = '../raw'

In [2]:
df = pd.read_excel(f'{rawpath}/Plakate_1933-45_ImportCBS.xlsx', names=['laendercode', 'entstehungszeit','titel','entstehungsort','konvolut4105','konvolut6710','signatur'])
df

,laendercode,entstehungszeit,titel,entstehungsort,konvolut4105,konvolut6710,signatur
0,/1XA-SUHH,1941-1942,WKII 15.6.3.1,Sowjetunion,!1180323777!,!1246329107!,WKII 15.6.3.1
1,/1XA-SUHH,1941-1942,WKII 15.6.3.2,Sowjetunion,!1180323777!,!1246329107!,WKII 15.6.3.2
2,/1XA-SUHH,1941-1942,WKII 15.6.3.3,Sowjetunion,!1180323777!,!1246329107!,WKII 15.6.3.3
3,/1XA-SUHH,1941-1942,WKII 15.6.3.4,Sowjetunion,!1180323777!,!1246329107!,WKII 15.6.3.4
4,/1XA-SUHH,1941-1942,WKII 15.6.3.5,Sowjetunion,!1180323777!,!1246329107!,WKII 15.6.3.5
...,...,...,...,...,...,...,...
1222,/1XA-BE/1XA-DXDE,1939-1943,WKII 15.3.1.655,Belgien,!117658510X!,!124632749X!,WKII 15.3.1.655
1223,/1XA-BE/1XA-DXDE,1939-1943,WKII 15.3.1.656,Belgien,!117658510X!,!124632749X!,WKII 15.3.1.656
1224,/1XA-BE/1XA-DXDE,1939-1943,WKII 15.3.1.657,Belgien,!117658510X!,!124632749X!,WKII 15.3.1.657
1225,/1XA-BE/1XA-DXDE,1939-1943,WKII 15.3.1.658,Belgien,!117658510X!,!124632749X!,WKII 15.3.1.658


In [3]:
def pica_schreiben(row):
    pica = f"""\t\n0500 Pa
0501 Text$btxt
0502 ohne Hilfsmittel zu benutzen$bn
0503 Blatt$bnb
0600 yy\n"""
    
    if len(str(row.entstehungszeit)) > 0 and len(str(row.entstehungszeit)) < 5:
        # nur ein datum
        pica += f"1100 {row.entstehungszeit}\n1110 *{row.entstehungszeit}$4ezth\n"
    elif len(str(row.entstehungszeit)) > 4 and len(str(row.entstehungszeit)) < 10:
        pica += f"1100 {row.entstehungszeit[:4]}$n[zwischen {row.entstehungszeit[:4]} und {row.entstehungszeit[5:]}]\n1110 zwischen {row.entstehungszeit[:4]} und {row.entstehungszeit[5:]}*{row.entstehungszeit}$a{row.entstehungszeit[:4]}$b{row.entstehungszeit[5:]}$4ezth\n"
        # wohl zwei daten
    else:
        pica += "\n\n1100 FEHLER\n\n"
    
    pica += f"""1130 TB-papier
1130 !040445224!
1131 !04046198X!
1132 a1-analog;a2-druck;f2-blatt
1500 /1und
1700 {row.laendercode}
2105 04,P01
4000 Plakat Nr. {row.titel} der Plakatsammlung 1933-1945
4019 Plakat Nr. {row.titel} der Plakatsammlung 1933-1945\n"""
    if pd.isna(row.entstehungsort):
        pica += "4030 [Ort nicht ermittelbar] : [Verlag nicht ermittelbar]\n"
    else:
        pica += f"4030 [{row.entstehungsort.strip()}] : [Verlag nicht ermittelbar]\n"
    
    pica += f"""4060 1 ungezähltes Blatt
4105 !118043062X!
4105 {row.konvolut4105}
4700 |BSM|
7001 {now.strftime('%d-%m-%y')} : x
6710 {row.konvolut6710.strip()}
6800 [Provenienz]
6800 !004709608! *1961-1993
6800 Stempel
7100 GS PL WKII @ m
7109 !!DBSM/S!! ; GS PL {row.signatur}
8034 Dieses Plakat ist Teil eines Erschließungsprojekts und derzeit nicht ausleihbar.\n"""
    return pica


In [4]:
now = datetime.now()
# testfile
with open(f"../dat/wkII-{now.strftime('%y-%m-%d-%H-%M-%S')}.dat","w") as f:
    with open("../dat/wkII-recent.dat","w") as f2:
        for index, row in df.iterrows():
            f.write(pica_schreiben(row))
            f2.write(pica_schreiben(row))

with open(f"../dat/wkII-recent-sample.dat","w") as f:
    for index, row in df.sample(5).iterrows():
        f.write(pica_schreiben(row))

In [5]:
now = datetime.now()
with open(f"../dat/wkII-final-appr.dat","w") as f:
    for index, row in df.sample(123).iterrows():
        f.write(pica_schreiben(row))